In [ ]:
import holodeck as holo
from holodeck import single_sources, utils, plot, detstats
from holodeck.constants import YR

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py



# Anisotropy Functions

In [ ]:
NSIDE = 24
LMAX = 8

def healpix_map(hc_ss, hc_bg, nside=NSIDE):
    """ Build mollview array of strains for a healpix map
    
    Parameters
    ----------
    hc_ss : (F,R,L) NDarray
        Characteristic strain of single sources.
    hc_bg : (F,R) NDarray
        Characteristic strain of the background.
    nside : integer
        number of sides for healpix map.

    Returns
    -------
    moll_hc : (NPIX,) 1Darray
        Array of strain at every pixel for a mollview healpix map.
    
    NOTE: Could speed up the for-loops, but it's ok for now.
    """

    npix = hp.nside2npix(nside)
    nfreqs = len(hc_ss)
    nreals = len(hc_ss[0])
    nloudest = len(hc_ss[0,0])

    # spread background evenly across pixels in moll_hc
    moll_hc = np.ones((nfreqs,nreals,npix)) * hc_bg[:,:,np.newaxis]/np.sqrt(npix) # (frequency, realization, pixel)

    # choose random pixels to place the single sources
    pix_ss = np.random.randint(0, npix-1, size=nfreqs*nreals*nloudest).reshape(nfreqs, nreals, nloudest)
    for ff in range(nfreqs):
        for rr in range(nreals):
            for ll in range(nloudest):
                moll_hc[ff,rr,pix_ss[ff,rr,ll]] = np.sqrt(moll_hc[ff,rr,pix_ss[ff,rr,ll]]**2
                                                          + hc_ss[ff,rr,ll]**2)
                
    return moll_hc

def sph_harm_from_map(moll_hc, lmax=LMAX):
    """ Calculate spherical harmonics from strains at every pixel of 
    a healpix mollview map.
    
    Parameters
    ----------
    moll_hc : (F,R,NPIX,) 1Darray
        Characteristic strain of each pixel of a healpix map.
    lmax : int
        Highest harmonic to calculate.

    Returns
    -------
    Cl : (F,R,lmax+1) NDarray
        Spherical harmonic coefficients 
        
    """
    nfreqs = len(moll_hc)
    nreals = len(moll_hc[0])

    Cl = np.zeros((nfreqs, nreals, lmax+1))
    for ff in range(nfreqs):
        for rr in range(nreals):
            Cl[ff,rr,:] = hp.anafast(moll_hc[ff,rr], lmax=lmax)

    return Cl

def sph_harm_from_hc(hc_ss, hc_bg, nside = NSIDE, lmax = LMAX):
    """ Calculate spherical harmonics and strain at every pixel
    of a healpix mollview map from single source and background char strains.

    Parameters
    ----------
    hc_ss : (F,R,L) NDarray
        Characteristic strain of single sources.
    hc_bg : (F,R) NDarray
        Characteristic strain of the background.
    nside : integer
        number of sides for healpix map.

    Returns
    -------
    moll_hc : (F,R,NPIX,) 2Darray
        Array of strain at every pixel for a mollview healpix map.
    Cl : (F,R,lmax+1) NDarray
        Spherical harmonic coefficients 
    
    """
    moll_hc = healpix_map(hc_ss, hc_bg, nside)
    Cl = sph_harm_from_map(moll_hc, lmax)

    return moll_hc, Cl



##########################################################################################
################################### PLOTTING FUNCTIONS ###################################
##########################################################################################

def plot_Cl(Cl, lvals, fobs_nHz, ff_labels=(0, 10, 20, 30, 39)):
    """ Plot C_l vs l for all frequencies of a single realization.
    
    """
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')
    colors=cm.rainbow(np.linspace(0,1,len(Cl)))

    for ff in range(len(Cl)):
        if ff in ff_labels:
            label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
        else: label = None
        ax.plot(lvals[:], Cl[ff,:], c=colors[ff], alpha=0.5, label=label,
                marker='o')
        
    ax.legend()
    return fig

def plot_ClC0(Cl, lvals, fobs_nHz, ff_labels=(0, 10, 20, 30, 39)):
    """ Plot C_l/C_0 for all frequencies of a single realization.
    
    """
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell/C_0$', xscale='linear')
    colors=cm.rainbow(np.linspace(0,1,len(Cl)))

    for ff in range(len(Cl)):
        if ff in ff_labels:
            label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
        else: label = None
        ax.plot(lvals[:], Cl[ff,:]/Cl[ff,0,np.newaxis], c=colors[ff], alpha=0.5, label=label,
                marker='o')
        
    ax.legend()
    return fig

def plot_llp1Cl(Cl, lvals, fobs_nHz, ff_labels=(0, 10, 20, 30, 39)):
    """ Plot l(l+1)C_l vs l for all frequencies of a single realization.
    
    """
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$\ell(\ell+1)C_\ell$', xscale='linear')
    colors=cm.rainbow(np.linspace(0,1,len(Cl)))

    xx = lvals[1:]
    yy = (lvals*(lvals+1))[np.newaxis,1:]*Cl[:,1:]
    for ff in range(len(Cl)):
        if ff in ff_labels:
            label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
        else: label = None
        ax.plot(xx, yy[ff], c=colors[ff], alpha=0.5, label=label,
                marker='o')
        
    ax.legend()
    return fig

def plot_Clg0(Cl, fobs):
    """ Plot C_l>0/C_0 vs f for many realizations.
    
    """

    fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$', xscale='linear')
    xx = fobs*YR
    print(xx.shape)
    print(np.sum(Cl[:,:,1:], axis=2).shape)
    print(Cl[:,:,0,np.newaxis].shape)
    yy = np.sum(Cl[:,:,1:], axis=2)/Cl[:,:,0]
    print(yy.shape)
    for rr in range(len(Cl[0])):
        ax.plot(xx, yy[:,rr], alpha=0.5, marker='o')

    ax.plot(xx,np.median(yy, axis=1), color='k')
        
    ax.legend()
    return fig

def map_from_sph_harm(Cl, nside=NSIDE, lmax=LMAX):
    nfreqs = len(Cl)
    nreals = len(Cl[0])
    npix = hp.nside2npix(nside)
    moll_sh = np.zeros((nfreqs,nreals,npix))
    for ff in range(nfreqs):
        for rr in range(nreals):
            moll_sh[ff,rr,:] = hp.synfast(Cl[ff,rr], nside, lmax=lmax)
    return moll_sh

# Ranking Sample Functions

In [ ]:
def amp_to_hc(amp_ref, fobs, dfobs):
    """ Calculate characteristic strain from strain amplitude.
    
    """
    hc = amp_ref*np.sqrt(fobs/dfobs)
    return hc

def rank_samples(hc_ss, hc_bg, fobs, dfobs=None, amp_ref=None, hc_ref=None, ret_all = False):
    """ Sort samples by those with f=1/yr char strains closest to some reference value.
    
    Parameters
    ----------
    hc_ss : (N,F,R,L) NDarray
        Characteristic strain of the loudest single sources.
    hc_bg : (N,F,R) NDarray
        Characteristic strain of the background.
    fobs : (F,) 
        Observed GW frequency
    dfobs : (F,) or None
        Observed GW frequency bin widths.
        only needed if using amp_ref
    amp_ref : scalar or None
        Reference strain amplitude at f=1/yr
    hc_ref : scalar or None
        Reference characteristic strain at f=1/yr
        Only one of hc_ref and amp_ref should be provided.


    Returns
    -------
    nsort : (N,) 1Darray
        Indices of the param space samples sorted by proximity to the reference 1yr amplitude.
    
    """
    
    # find frequency bin nearest to 1/10yr
    fidx = (np.abs(fobs - 1/(10*YR))).argmin()

    if (hc_ref is None):
        # find reference (e.g. 12.5 yr) char strain
        hc_ref = amp_to_hc(amp_ref, fobs[fidx], dfobs[fidx])
        
    # select 1/yr median strains of samples
    hc_tt = np.sqrt(hc_bg[:,fidx,:]**2 + np.sum(hc_ss[:,fidx,:,:]**2, axis=-1)) # (N,R)
    hc_tt = np.median(hc_tt, axis=1) 

    # extrapolate hc_ref at freq closest to 1/10yr from 1/10yr ref
    hc_ref = hc_ref * (fobs[fidx]*YR/.1)**(-2/3)

    # sort by closest
    nsort = np.argsort(np.abs(hc_tt-hc_ref))

    if ret_all:
        return nsort, fidx, hc_tt, hc_ref
    return nsort

# Read in Library

ss16 has 10 loudest, definitely will replace this with a better library

In [ ]:
sspath = '/Users/emigardiner/GWs/holodeck/output/2023-05-16-mbp-ss18_n100_r40_d15_f30_l1000_p0/'
hdfname = sspath+'ss_lib.hdf5'
ssfile = h5py.File(hdfname, 'r')
print(list(ssfile.keys()))
hc_ss = ssfile['hc_ss'][...]
hc_bg = ssfile['hc_bg'][...]
fobs = ssfile['fobs'][:]
dfobs = ssfile['dfobs'][:]
ssfile.close()

shape = hc_ss.shape
nsamps, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
print('N,F,R,L =', nsamps, nfreqs, nreals, nloudest)

# Find Best Samples

In [ ]:
hc_ref15_10yr = 11.2*10**-15 
nsort, fidx, hc_tt, hc_ref15 = rank_samples(hc_ss, hc_bg, fobs, hc_ref=hc_ref15_10yr, ret_all=True)
nbest = 10
print(hc_ref15)


### Plot amplitudes sorted

In [ ]:
# amplitudes from sorting
fig, ax = plot.figax(figsize=(7,5), xlabel='Sample', ylabel='$h_{c,\mathrm{tot}}(f=1/\mathrm{yr})$',
                     xscale='linear')

ax.scatter(nsort[:nbest], hc_tt[nsort][:nbest], s=20, marker='o', c=np.arange(len(nsort))[:nbest], 
           cmap='jet_r', label='%d best samples' % nbest)
ax.scatter(nsort[nbest:], hc_tt[nsort][nbest:], s=15, marker='x', c='darkblue', 
           label='excluded samples')
ax.axhline(hc_ref15, label='15 yr ref: %.2e' % (hc_ref15), linestyle='dashed', color='tab:red', alpha=0.5)
# plt.scatter(np.arange(nsamps), hc_tt[nsort], s=3, label= r'15 yr ref: $A=$' + '%.2f' % amp_ref15)

ax.set_yscale('log')
ax.set_xlabel('Samples')
ax.set_ylabel('$h_c(f=%.2f\mathrm{yr}^{-1})$' % (fobs[fidx]*YR))
ax.legend()

In [ ]:
# amplitudes from sorting
fig, ax = plot.figax(figsize=(7,5), xlabel='Sample', ylabel='$h_{c,\mathrm{tot}}(f=1/\mathrm{yr})$',
                     xscale='linear')
ax.scatter(nsort, hc_tt[nsort], s=20, marker='o', c=np.arange(len(nsort)), 
           cmap='jet_r')
ax.axhline(hc_ref15, label='15 yr ref: %.2e' % (hc_ref15), linestyle='dashed', color='tab:red', alpha=0.5)
# plt.scatter(np.arange(nsamps), hc_tt[nsort], s=3, label= r'15 yr ref: $A=$' + '%.2f' % amp_ref15)

ax.set_yscale('log')
ax.set_xlabel('Samples')
ax.set_ylabel('$h_c(f=%.2f\mathrm{yr}^{-1})$' % (fobs[fidx]*YR))
ax.legend()

### Plot strains from best samples

In [ ]:
print(nsort[:5])
xx = fobs*YR
yy = hc_ref15_10yr * (xx/.1)**(-2/3)
for nn in nsort[:5]:
    print('nn=%d' % nn)
    fig = plot.plot_bg_ss(fobs, hc_bg[nn], hc_ss[nn,:,:,:100]) # just 100 loudest, cant see more anyway
    ax = fig.axes[0]
    ax.plot(xx,yy,linestyle='dashdot', color='grey', alpha=1, 
            label= '$(%.2e) [f/(.1 \mathrm{yr^-1})]^{-2/3}  $' % hc_ref15_10yr)
    ax.axhline(hc_ref15, label='15 yr ref: $h_c(%.2f/\mathrm{yr}) = %.2e$' % (xx[fidx], hc_ref15), linestyle='dashed', color='tab:red', alpha=0.5)
    ax.axvline(xx[fidx], color='k', alpha=0.5, linestyle='dashed', label='$f=%.2f\mathrm{yr}$' % xx[fidx])
    ax.set_title('Sample %d' % nn)
    ax.legend(loc='lower left')

# Calculate pixel strains and harmonic coefficients

In [ ]:
nside = NSIDE
npix = hp.nside2npix(nside)
lmax = LMAX
nbest = 10
Cl_best = np.zeros((nbest, nfreqs, nreals, lmax+1 ))
moll_hc_best = np.zeros((nbest, nfreqs, nreals, npix))
for nn in range(nbest):
    print('on nn=%d out of nbest=%d' % (nn,nbest))
    moll_hc_best[nn,...], Cl_best[nn,...] = sph_harm_from_hc(hc_ss[nsort[nn]], 
                                            hc_bg[nsort[nn]],
                                            nside=nside, lmax=lmax)

### Plot Results

In [ ]:
lvals = np.arange(lmax+1)
fobs_nHz = fobs*10**9
fobs_yrs = fobs*YR
ff_labels = (0,9,19,29,39)

#### Plot_Cl_medians

In [ ]:
def plot_Cl_medians(lvals, Cl_best, lmax):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')

    xx = lvals # (l)
    yy = Cl_best # (B,F,R,l)
    yy = np.median(yy, axis=2) # (B,F,l)

    colors = cm.rainbow(np.linspace(1, 0, nfreqs+1))
    rr=0
    for ff in (np.arange(0,nfreqs,8)):
        label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
        ax.plot(xx, np.median(yy[:,ff,:], axis=0), color=colors[ff], label=label,
                linewidth=3)
        for pp in [50, 98]:
            percs = pp/2
            percs = [50-percs, 50+percs]
            ax.fill_between(xx, *np.percentile(yy[:,ff,:], percs, axis=0), alpha=0.25, color=colors[ff])
        for bb in range(nbest):
            ax.plot(xx, yy[bb,ff,:], color=colors[ff], linestyle=':', alpha=0.75)

    ax.legend()
    ax.set_title('50%% and 98%% confidence intervals of the %d best samples \nusing realizations medians, lmax=%d'
                % (nbest, lmax))
    return fig

fig = plot_Cl_medians(lvals, Cl_best, lmax)

In [ ]:
fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')

xx = lvals # (l)
yy = Cl_best # (B,F,R,l)
yy = np.median(yy, axis=2) # (B,F,l)

# choose a frequency 
ff = fidx # f=1/yr
label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
yy = yy[:,ff,:] # (B,l)

colors = cm.rainbow(np.linspace(1, 0, 31))
ax.plot(xx, np.median(yy, axis=0), color='g', label=label, lw='3')
for pp in [50, 98]:
    percs = pp/2
    percs = [50-percs, 50+percs]
    ax.fill_between(xx, *np.percentile(yy, percs, axis=0), alpha=0.25, color='g')

colors = cm.rainbow(np.linspace(0,1,nbest))
for bb in range(nbest):
    ax.plot(xx, yy[bb,:], color=colors[bb], linestyle=':', alpha=0.75)
ax.legend()
ax.set_title('50%% and 98%% Confidence Intervals of the %d Best Samples, \nusing median among realizations, lmax=%d' % (nbest, lmax))

#### plot_ClC0_medians

In [ ]:
def plot_ClC0_medians(lvals, Cl_best, lmax):
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell/C_0$', xscale='linear')

    xx = lvals[1:] # (l)
    yy = Cl_best[:,:,:,1:]/Cl_best[:,:,:,0,np.newaxis] # (B,F,R,l)
    yy = np.median(yy, axis=2) # (F,R,l)

    colors = cm.rainbow(np.linspace(1, 0, nfreqs+1))
    rr=0
    for ff in (np.arange(0,nfreqs,8)):
        label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
        ax.plot(xx, np.median(yy[:,ff,:], axis=0), color=colors[ff], label=label,
                linewidth=4)
        for pp in [50, 98]:
            percs = pp/2
            percs = [50-percs, 50+percs]
            ax.fill_between(xx, *np.percentile(yy[:,ff,:], percs, axis=0), alpha=0.25, color=colors[ff],
                            linestyle='solid', linewidth=3)
        for bb in range(nbest):
            ax.plot(xx, yy[bb,ff,:], color=colors[ff], linestyle=':', alpha=0.75)

    fig.legend(ncol=3, bbox_to_anchor=(-.01,-.27), 
            bbox_transform=ax.transAxes, loc='lower left')
    ax.set_title('50%% and 98%% confidence intervals of the %d best samples \nusing realizations medians, lmax=%d'
                % (nbest, lmax))
    fig.tight_layout()
    return fig

fig = plot_ClC0_medians(lvals, Cl_best, lmax)

In [ ]:
fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell/C_0$', xscale='linear')

xx = lvals[1:] # (l)
yy = Cl_best[:,:,:,1:]/Cl_best[:,:,:,0,np.newaxis] # (B,F,R,l)
yy = np.median(yy, axis=2) # (B,F,l)

# choose a frequency 
ff = fidx # f=1/yr
label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
yy = yy[:,ff,:] # (B,l)

colors = cm.rainbow(np.linspace(1, 0, 31))
ax.plot(xx, np.median(yy, axis=0), color='g', label=label, lw=4)
for pp in [50, 98]:
    percs = pp/2
    percs = [50-percs, 50+percs]
    ax.fill_between(xx, *np.percentile(yy, percs, axis=0), alpha=0.25, color='g')

colors = cm.rainbow(np.linspace(0,1,nbest))
for bb in range(nbest):
    ax.plot(xx, yy[bb,:], color=colors[bb], linestyle=':', alpha=0.75)
ax.legend()
ax.set_title('50%% and 98%% Confidence Intervals of the %d Best Samples, \nusing median among realizations' % (nbest))

#### plot_Clg0_medians

In [ ]:
def plot_Clg0_medians(Cl_best, lmax):
    fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$')

    xx = fobs_yrs
    yy = np.sum(Cl_best[:,:,:,1:], axis=-1)/Cl_best[:,:,:,0,] # (B,F,R)
    yy = np.median(yy, axis=-1) # (B,F) median over realizations

    ax.plot(xx, np.median(yy, axis=0), color='k')
    for pp in [50, 98]:
        percs = pp/2
        percs = [50-percs, 50+percs]
        ax.fill_between(xx, *np.percentile(yy, percs, axis=0), alpha=0.2, color='k')

    colors = cm.rainbow(np.linspace(1, 0, nbest))
    for bb in range(nbest):
        ax.plot(xx, yy[bb,:], color=colors[bb], linestyle=':', alpha=0.75,
                linewidth=2)
    ax.legend()
    ax.set_title('50%% and 98%% confidence intervals of the %d best samples \nusing realizations medians, lmax=%d'
                % (nbest, lmax))
    return fig
fig = plot_Clg0_medians(Cl_best, lmax)

In [ ]:
fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$')

xx = fobs_yrs
yy = np.sum(Cl_best[:,:,:,1:], axis=-1)/Cl_best[:,:,:,0,] # (B,F,R)

colors = cm.rainbow(np.linspace(1, 0, 5))
rr=0
for bb in range(5):
    # label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
    label='p = %d' % nsort[bb]
    ax.plot(xx, np.median(yy[bb,:,:], axis=-1), color=colors[bb], label=label)
    for pp in [50, 98]:
        percs = pp/2
        percs = [50-percs, 50+percs]
        ax.fill_between(xx, *np.percentile(yy[bb,:,:], percs, axis=-1), alpha=0.25, color=colors[bb])
ax.legend()
ax.set_title('50%% and 98%% confidence intervals from %d realizations of the %d best samples' % (nreals,5))

# Look at 1 sample, more closely

In [ ]:
bb = 0 # best sample
nn = nsort[bb]
rr = 0
print(Cl_best.shape)
print(nn)

In [ ]:
fig1 = plot_Cl(Cl_best[bb,:,rr,:], lvals, fobs_nHz, ff_labels=(0,9,19,29,39))
fig1.axes[0].set_title('Sample %d, Realization %d' % (nn,rr))

fig2 = plot_ClC0(Cl_best[bb,:,rr,:], lvals, fobs_nHz, ff_labels=(0,9,19,29,39))
fig2.axes[0].set_title('Sample %d, Realization %d' % (nn,rr))

fig3 = plot_llp1Cl(Cl_best[bb,:,rr,:], lvals, fobs_nHz, ff_labels=(0,9,19,29,39))
fig3.axes[0].set_title('Sample %d, Realization %d' % (nn,rr))

fig4 = plot_Clg0(Cl_best[bb,:,:,:], fobs)

In [ ]:
fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$')

xx = fobs_yrs
yy = np.sum(Cl_best[:,:,:,1:], axis=-1)/Cl_best[:,:,:,0,] # (B,F,R)

colors = cm.rainbow(np.linspace(1, 0, 5))
bb=0
    # label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
ax.plot(xx, np.median(yy[bb,:,:], axis=1), color='k')
for pp in [50, 98]:
    percs = pp/2
    percs = [50-percs, 50+percs]
    ax.fill_between(xx, *np.percentile(yy[bb,:,:], percs, axis=1), alpha=0.15, color='k')

for rr in range(20):
    ax.plot(xx, yy[bb,:,rr], linestyle=':', alpha=0.75)
ax.legend()
ax.set_title('50%% and 98%% confidence intervals from %d realizations of the %dth samples' % (nreals,nsort[bb]))

# Repeat for lmax=6

In [ ]:
nside = NSIDE
npix = hp.nside2npix(nside)
lmax6 = 6
lvals6 = np.arange(lmax6+1)
nbest = 10
Cl_best6 = np.zeros((nbest, nfreqs, nreals, lmax6+1 ))
moll_hc_best6 = np.zeros((nbest, nfreqs, nreals, npix))
for nn in range(nbest):
    print('on nn=%d out of nbest=%d' % (nn,nbest))
    moll_hc_best6[nn,...], Cl_best6[nn,...] = sph_harm_from_hc(hc_ss[nsort[nn]], 
                                            hc_bg[nsort[nn]],
                                            nside=nside, lmax=lmax6)

### Plot Results

In [ ]:
fig1 = plot_Cl_medians(lvals6, Cl_best6, lmax6)
fig2 = plot_ClC0_medians(lvals6, Cl_best6, lmax6)
fig3 = plot_Clg0_medians(Cl_best6, lmax6)

In [ ]:
fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$')

xx = fobs_yrs
yy = np.sum(Cl_best6[:,:,:,1:], axis=-1)/Cl_best6[:,:,:,0,] # (B,F,R)

colors = cm.rainbow(np.linspace(1, 0, 5))
bb=0
    # label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
ax.plot(xx, np.median(yy[bb,:,:], axis=1), color='k')
for pp in [50, 98]:
    percs = pp/2
    percs = [50-percs, 50+percs]
    ax.fill_between(xx, *np.percentile(yy[bb,:,:], percs, axis=1), alpha=0.15, color='k')

for rr in range(20):
    ax.plot(xx, yy[bb,:,rr], linestyle=':', alpha=0.75)
ax.legend()
ax.set_title('50%% and 98%% confidence intervals from %d realizations of the %dth samples\nlmax=%d' % (nreals,nsort[bb], lmax6))

# Using 12.5 yr data

In [ ]:
# amp_ref12 = 1.53*10**-15
# nsort12, fidx12, hc_tt12 = rank_samples(hc_ss, hc_bg, fobs, dfobs, amp_ref=amp_ref12, ret_all=True)
# nbest = 25
# hc_ref12 = amp_ref12 * np.sqrt(fobs[fidx]/dfobs[fidx])
# print(hc_ref12)


In [ ]:
# # amplitudes from sorting
# fig, ax = plot.figax(figsize=(7,5), xlabel='Sample', ylabel='$h_{c,\mathrm{tot}}(f=1/\mathrm{yr})$',
#                      xscale='linear')
# ax.scatter(nsort12, hc_tt[nsort12], s=20, marker='o', c=np.arange(len(nsort)), 
#            cmap='jet_r')
# ax.axhline(hc_ref12, label='12.5 yr ref: %.2e' % (hc_ref12), linestyle='dashed', color='tab:red', alpha=0.5)
# # plt.scatter(np.arange(nsamps), hc_tt[nsort], s=3, label= r'15 yr ref: $A=$' + '%.2f' % amp_ref15)

# plt.yscale('log')
# plt.xlabel('Samples')
# plt.ylabel('$h_c(f=1\mathrm{yr}^{-1})$')
# plt.legend()

In [ ]:
# print(nsort12[:5])
# for nn in nsort12[:5]:
#     print('nn=%d' % nn)
#     fig = plot.plot_bg_ss(fobs, hc_bg[nn], hc_ss[nn,:,:,:100]) # just 100 loudest, cant see more anyway
#     ax = fig.axes[0]
#     ax.axhline(hc_ref12, label='12.5 yr ref: %.2e' % (hc_ref12),
#                 linestyle='dashed', color='tab:red', alpha=0.5)

#     ax.axvline(1/10, color='k', alpha=0.5, linestyle='dashed', label='$f=1/10\mathrm{yr}$')
#     ax.set_title('Sample %d' % nn)
#     ax.legend()

### Maps for funsies

In [ ]:
rr=0
moll_hc = moll_hc_best[0]
F=nfreqs
L=nloudest

fig, axs = plt.subplots(nrows=10, ncols=int(F/10))
for ff in range(F):
    title='$f$= %.2f yr$^{-1}$ (bin %d/%d)' % (fobs[ff]*YR, ff, F)
    row, col = int(ff/3), ff%3
    # print(row,col)
    plt.axes(axs[row,col])
    hp.mollview(moll_hc[ff,rr], title=title, hold=True)

fig.suptitle('%d Loudest Single Sources' % L, y=.895)
fig.tight_layout()
fig.set_size_inches(18, 30)
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

In [ ]:
Cl = Cl_best[0]

moll_sh = map_from_sph_harm(Cl)

rr=0
fig, axs = plt.subplots(nrows=10, ncols=int(F/10))
for ff in range(F):
    title='$f$= %.2f yr$^{-1}$ (bin %d/%d)' % (fobs[ff]*YR, ff, F)
    row, col = int(ff/3), ff%3
    # print(row,col)
    plt.axes(axs[row,col])
    hp.mollview(moll_sh[ff,rr], title=title, hold=True)

fig.suptitle('%d Loudest Single Sources, Recovered from Spherical Harmonics' % L, y=.895)
fig.tight_layout()
fig.set_size_inches(18, 30)
# fig.savefig(saveloc+'all_freqs.png', dpi=1000)

# fig

In [ ]:
ff=fidx
title = '$f$= %.2f yr$^{-1}$ (bin %d/%d)' % (fobs[ff]*YR, ff, F)
hp.mollview(moll_hc[ff,rr], title='Randomly Placed: '+title, hold=False)
hp.mollview(moll_sh[ff,rr], title='Reconstructed from Harmonics: '+title, hold=False)


In [ ]:
print(npix)